In [1]:
import os
import sys

sys.path.insert(0, '..')

import pyarrow.parquet as pq

from pysalient import io as io
from pysalient import visualisation as vis
from pysalient.evaluation import evaluation
from pysalient.evaluation import compare_models

In [2]:
sample_data_path = os.path.join("data", "anonymised_sample.parquet")
# count rows
table = pq.read_table(sample_data_path)
print(f"Number of rows: {table.num_rows}")
# print column names
print(table.column_names)
# Convert the 'true_label' column to a pandas Series
true_label_series = table["true_label"].to_pandas()

# Count the number of true labels (1)
true_count = (true_label_series == 1).sum()

print(f"Number of true labels (1): {true_count}")

# Convert the table to a pandas DataFrame for easier grouping
df = table.to_pandas()

# show table
print(df.head(5))


grouped = df.groupby("encounter_id")

# Count the number of unique groups (encounters)
num_groups = df["encounter_id"].nunique()
print(f"Number of unique encounter groups: {num_groups}")

# Calculate the sum of 'true_label' for each group
group_sums = grouped["true_label"].sum()

# Count how many groups have at least one true positive (sum > 0)
groups_with_positives = (group_sums > 0).sum()
print(
    f"Number of encounter groups with at least one true positive: {groups_with_positives}"
)

Number of rows: 21356
['encounter_id', 'event_timestamp', 'culture_event', 'suspected_infection', 'true_label', 'prediction_proba_1', 'prediction_proba_2']
Number of true labels (1): 789
  encounter_id  event_timestamp  culture_event  suspected_infection  \
0     0666505c              2.0            1.0                  1.0   
1     0666505c              2.0            1.0                  1.0   
2     0666505c              3.0            1.0                  1.0   
3     0666505c              3.0            1.0                  1.0   
4     0666505c              4.0            1.0                  1.0   

   true_label  prediction_proba_1  prediction_proba_2  
0           1            0.185334            0.531925  
1           1            0.185334            0.531925  
2           1            0.134316            0.360046  
3           1            0.134316            0.360046  
4           1            0.118005            0.167655  
Number of unique encounter groups: 100
Number of e

In [3]:
sample_data_path = os.path.join("data", "anonymised_sample.parquet")
# count rows
table = pq.read_table(sample_data_path)
print(f"Number of rows: {table.num_rows}")
# print column names
print(table.column_names)
# Convert the 'true_label' column to a pandas Series
true_label_series = table["true_label"].to_pandas()

# Count the number of true labels (1)
true_count = (true_label_series == 1).sum()

print(f"Number of true labels (1): {true_count}")

# Convert the table to a pandas DataFrame for easier grouping
df = table.to_pandas()

# show table
print(df.head(5))


grouped = df.groupby("encounter_id")

# Count the number of unique groups (encounters)
num_groups = df["encounter_id"].nunique()
print(f"Number of unique encounter groups: {num_groups}")

# Calculate the sum of 'true_label' for each group
group_sums = grouped["true_label"].sum()

# Count how many groups have at least one true positive (sum > 0)
groups_with_positives = (group_sums > 0).sum()
print(
    f"Number of encounter groups with at least one true positive: {groups_with_positives}"
)

Number of rows: 21356
['encounter_id', 'event_timestamp', 'culture_event', 'suspected_infection', 'true_label', 'prediction_proba_1', 'prediction_proba_2']
Number of true labels (1): 789
  encounter_id  event_timestamp  culture_event  suspected_infection  \
0     0666505c              2.0            1.0                  1.0   
1     0666505c              2.0            1.0                  1.0   
2     0666505c              3.0            1.0                  1.0   
3     0666505c              3.0            1.0                  1.0   
4     0666505c              4.0            1.0                  1.0   

   true_label  prediction_proba_1  prediction_proba_2  
0           1            0.185334            0.531925  
1           1            0.185334            0.531925  
2           1            0.134316            0.360046  
3           1            0.134316            0.360046  
4           1            0.118005            0.167655  
Number of unique encounter groups: 100
Number of e

In [4]:
# Define the path relative to the project root
# Assuming the notebook is run from the project root or examples/ directory
sample_data_path = os.path.join("data", "anonymised_sample.parquet")

assigned_table_events = None

if os.path.exists(sample_data_path):
    # Use the actual column names identified during inspection directly
    # Ensure these names actually exist based on the printout above!
    model_1_evaluation = io.load_evaluation_data(
        source=sample_data_path,
        y_proba_col="prediction_proba_1",
        y_label_col="true_label",
        aggregation_cols=None,
        timeseries_col="event_timestamp",
        # We don't provide task_col or model_col from the source
        # assign_task_name="AKI",  # Assign this name to the new 'task' column
        # assign_model_name="LogRegress",  # Assign this name to the new 'model' column
    )

    print("\nSuccessfully loaded data with assigned names (Model 1):")
    print(model_1_evaluation.schema)
    print(f"\nNumber of rows: {model_1_evaluation.num_rows}")

    # Display first few rows to verify new columns
    print("\nFirst 5 rows (with added 'task' and 'model' columns):")
    print(model_1_evaluation.slice(0, 5).to_pandas())
    model_2_evaluation = io.load_evaluation_data(
        source=sample_data_path,
        y_proba_col="prediction_proba_2",
        y_label_col="true_label",
        aggregation_cols=None,
        timeseries_col="event_timestamp",
        # We don't provide task_col or model_col from the source
        # assign_task_name="AKI",  # Assign this name to the new 'task' column
        # assign_model_name="LogRegress",  # Assign this name to the new 'model' column
    )
    print("\nSuccessfully loaded data with assigned names (Model 1):")
    print(model_2_evaluation.schema)
    print(f"\nNumber of rows: {model_2_evaluation.num_rows}")

    # Display first few rows to verify new columns
    print("\nFirst 5 rows (with added 'task' and 'model' columns):")
    print(model_2_evaluation.slice(0, 5).to_pandas())

else:
    print(
        f"Skipping data loading as file was not found: {sample_data_path}"
    )



Successfully loaded data with assigned names (Model 1):
encounter_id: string
event_timestamp: double
culture_event: double
suspected_infection: double
true_label: int64
prediction_proba_1: float
prediction_proba_2: float
-- schema metadata --
pysalient.io.y_proba_col: 'prediction_proba_1'
pysalient.io.y_label_col: 'true_label'
pysalient.io.timeseries_col: 'event_timestamp'
pysalient.io.aggregation_cols: '[]'

Number of rows: 21356

First 5 rows (with added 'task' and 'model' columns):
  encounter_id  event_timestamp  culture_event  suspected_infection  \
0     0666505c              2.0            1.0                  1.0   
1     0666505c              2.0            1.0                  1.0   
2     0666505c              3.0            1.0                  1.0   
3     0666505c              3.0            1.0                  1.0   
4     0666505c              4.0            1.0                  1.0   

   true_label  prediction_proba_1  prediction_proba_2  
0           1            0

In [5]:
# Define evaluation parameters
eval_modelid_1= "LogRegress_01"  # Use a generic ID as model wasn't assigned here
eval_modelid_2= "LightGBM_01"  # Use a generic ID as model wasn't assigned here
eval_filter = "ExampleFilterDummy"  # Describe the data subset
eval_thresholds = (0.1, 0.9, 0.1)  # Range: 0.1, 0.2, ..., 0.9
# eval_thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] # Example: List of thresholds

# Run the evaluation
evaluation_results_1 = evaluation(
    data=model_1_evaluation,  # Use the table loaded with col_map
    modelid=eval_modelid_1,
    filter_desc=eval_filter,
    thresholds=eval_thresholds,
    decimal_places=3,  # Control rounding of output floats # check that -1 is no rounding.
    calculate_au_ci=True,  # Enable AU CI calculation (uses bootstrap)
    calculate_threshold_ci=True,
    threshold_ci_method="bootstrap",  # Method for threshold CIs (ignored if calculate_threshold_ci=False)
    ci_alpha=0.05,  # 95% CI
    bootstrap_seed=42,  # For reproducible CIs
    bootstrap_rounds=1000,  # Fewer rounds for notebook speed
    force_threshold_zero=True,
    verbosity=1,
)
evaluation_results_2 = evaluation(
    data=model_2_evaluation,  # Use the table loaded with col_map
    modelid=eval_modelid_2,
    filter_desc=eval_filter,
    thresholds=eval_thresholds,
    decimal_places=3,  # Control rounding of output floats # check that -1 is no rounding.
    calculate_au_ci=True,  # Enable AU CI calculation (uses bootstrap)
    calculate_threshold_ci=True,
    threshold_ci_method="bootstrap",  # Method for threshold CIs (ignored if calculate_threshold_ci=False)
    ci_alpha=0.05,  # 95% CI
    bootstrap_seed=42,  # For reproducible CIs
    bootstrap_rounds=1000,  # Fewer rounds for notebook speed
    force_threshold_zero=True,
    verbosity=1,
)

/mnt/e/dev/pysalient/.venv/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
Metric func '_calculate_npv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_ppv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_ppv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_ppv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_pp

In [6]:
model_compare = compare_models(
    evaluation_results=[evaluation_results_1,evaluation_results_2],
    model_labels=['lg','lgm'],
    include_metrics=['AUROC','AUPRC']
    )

In [7]:
# Visualisation
styled_results = vis.format_evaluation_table(
    evaluation_results_1, decimal_places=3, ci_column=False
)
display(styled_results)

,modelid,filter_desc,threshold,time_to_first_alert_value,time_to_first_alert_unit,AUROC,AUPRC,Prevalence,Sample_Size,Label_Count,TP,TN,FP,FN,PPV,Sensitivity,Specificity,NPV,Accuracy,F1_Score
0,LogRegress_01,ExampleFilterDummy,0.000,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,789,0,20567,0,0.037 [0.034 - 0.039],1.000 [1.000 - 1.000],0.000 [0.000 - 0.000],0.000,0.037 [0.034 - 0.039],0.071 [0.067 - 0.076]
1,LogRegress_01,ExampleFilterDummy,0.100,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,348,17957,2610,441,0.118 [0.106 - 0.129],0.441 [0.406 - 0.476],0.873 [0.869 - 0.878],0.976 [0.974 - 0.978],0.857 [0.852 - 0.862],0.186 [0.169 - 0.203]
2,LogRegress_01,ExampleFilterDummy,0.200,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,70,20416,151,719,0.317 [0.254 - 0.377],0.089 [0.069 - 0.108],0.993 [0.991 - 0.994],0.966 [0.964 - 0.968],0.959 [0.957 - 0.962],0.139 [0.109 - 0.168]
3,LogRegress_01,ExampleFilterDummy,0.300,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20565,2,789,0.000 [0.000 - 0.000],0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.965],0.000 [0.000 - 0.000]
4,LogRegress_01,ExampleFilterDummy,0.400,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
5,LogRegress_01,ExampleFilterDummy,0.500,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
6,LogRegress_01,ExampleFilterDummy,0.600,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
7,LogRegress_01,ExampleFilterDummy,0.700,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
8,LogRegress_01,ExampleFilterDummy,0.800,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
9,LogRegress_01,ExampleFilterDummy,0.900,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]


In [8]:
# Visualisation
styled_results = vis.format_evaluation_table(
    evaluation_results_2, decimal_places=3, ci_column=False
)
display(styled_results)

,modelid,filter_desc,threshold,time_to_first_alert_value,time_to_first_alert_unit,AUROC,AUPRC,Prevalence,Sample_Size,Label_Count,TP,TN,FP,FN,PPV,Sensitivity,Specificity,NPV,Accuracy,F1_Score
0,LightGBM_01,ExampleFilterDummy,0.000,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,789,0,20567,0,0.037 [0.034 - 0.039],1.000 [1.000 - 1.000],0.000 [0.000 - 0.000],0.000,0.037 [0.034 - 0.039],0.071 [0.067 - 0.076]
1,LightGBM_01,ExampleFilterDummy,0.100,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,345,18205,2362,444,0.127 [0.114 - 0.140],0.437 [0.405 - 0.472],0.885 [0.881 - 0.889],0.976 [0.974 - 0.978],0.869 [0.864 - 0.873],0.197 [0.179 - 0.215]
2,LightGBM_01,ExampleFilterDummy,0.200,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,70,20368,199,719,0.260 [0.209 - 0.315],0.089 [0.070 - 0.110],0.990 [0.989 - 0.992],0.966 [0.964 - 0.968],0.957 [0.954 - 0.960],0.132 [0.105 - 0.161]
3,LightGBM_01,ExampleFilterDummy,0.300,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,25,20534,33,764,0.431 [0.300 - 0.569],0.032 [0.020 - 0.046],0.998 [0.998 - 0.999],0.964 [0.962 - 0.967],0.963 [0.960 - 0.965],0.059 [0.038 - 0.084]
4,LightGBM_01,ExampleFilterDummy,0.400,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,8,20561,6,781,0.571 [0.312 - 0.846],0.010 [0.004 - 0.018],1.000 [0.999 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.020 [0.008 - 0.034]
5,LightGBM_01,ExampleFilterDummy,0.500,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,3,20567,0,786,1.000 [1.000 - 1.000],0.004 [0.000 - 0.009],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.008 [0.000 - 0.017]
6,LightGBM_01,ExampleFilterDummy,0.600,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
7,LightGBM_01,ExampleFilterDummy,0.700,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
8,LightGBM_01,ExampleFilterDummy,0.800,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]
9,LightGBM_01,ExampleFilterDummy,0.900,nan,None,0.794 [0.779 - 0.810],0.161 [0.139 - 0.186],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000]


In [9]:
styled_results = vis.format_evaluation_table(
    model_compare, decimal_places=3, ci_column=False
)
display(styled_results)

,threshold,model,metric,value,lower_ci,upper_ci
0,0.000,lg,AUROC,0.706000,0.682000,0.727000
1,0.100,lg,AUROC,0.706000,0.682000,0.727000
2,0.200,lg,AUROC,0.706000,0.682000,0.727000
3,0.300,lg,AUROC,0.706000,0.682000,0.727000
4,0.400,lg,AUROC,0.706000,0.682000,0.727000
5,0.500,lg,AUROC,0.706000,0.682000,0.727000
6,0.600,lg,AUROC,0.706000,0.682000,0.727000
7,0.700,lg,AUROC,0.706000,0.682000,0.727000
8,0.800,lg,AUROC,0.706000,0.682000,0.727000
9,0.900,lg,AUROC,0.706000,0.682000,0.727000
